# Insert Cursor Demo

- Shows how to use an insert cursor
- Goal: Take a table (csv file) and make it spatial!

In [ ]:
import os
from arcpy import da
from arcpy import env
import arcpy

In [ ]:
import pandas as pd

In [ ]:
# Monument data from a github site. 
df = pd.read_csv("https://raw.githubusercontent.com/mike-n-7/ML4/master/data/monuments.csv")
df.head()

### Create a Feature Class

##### Check if the Feature Class Exists

- recall: When creating feature classes, spatial tables not exist

In [ ]:
fc = os.path.join(env.scratchGDB, "monuments")
if arcpy.Exists(fc):
    arcpy.management.Delete(fc)

##### Create Spatial Reference

In [ ]:
sr = arcpy.SpatialReference(4326)

#### Create The Feature Class 

In [ ]:
fc = arcpy.management.CreateFeatureclass(out_name=os.path.basename(fc), 
                                         out_path=os.path.dirname(fc), 
                                         geometry_type="POINT", 
                                         spatial_reference=sr)[0]
print(fc)

#### Add Fields to the Feature Class

- Use the `arcpy.management.AddField()`

In [ ]:
arcpy.management.AddField(in_table=fc, field_name="NOM", field_type="TEXT")
arcpy.management.AddField(in_table=fc, field_name="LONG", field_type="DOUBLE")
arcpy.management.AddField(in_table=fc, field_name="LAT", field_type="DOUBLE")
fields = [fld.name for fld in arcpy.ListFields(fc) \
          if fld.type != "Geometry"]
fields

#### Load the Data

- Use an Insert Cursor to add the information


In [ ]:
fields = [fld.name for fld in arcpy.ListFields(fc) \
          if fld.type not in  ["Geometry", "OID"]]
fields += ['Shape@']

In [ ]:
fields

**A Sample of the Data**

In [ ]:
dict(zip(df.columns.tolist(), df.values[0].tolist()))

### Create the Cursor Object

- Use the `with` pattern to create a cursor
- For each row in the Data Frame, **insert** a new row
- Finally clean up the cursor and release the lock
    + `del` ensure the lock is removed (though should not be needed)

In [ ]:
with da.InsertCursor(fc, fields) as icur:
    for row in df.values:
        pt = arcpy.Point(row[-2], row[-1])
        ptgeom = arcpy.PointGeometry(pt, sr)
        icur.insertRow([row[1], row[-2], row[-1], ptgeom])
        
del icur

##### Display the Added Rows

In [ ]:
arcpy.management.GetCount(fc)[0]

### Map It

In [ ]:
from arcgis.gis import GIS
gis = GIS()

In [ ]:
m1 = gis.map()
m1.center = (45.4944962453396,-73.53701983501136)
m1

In [ ]:
m1.zoom = 12

In [ ]:
from arcgis.features import GeoAccessor, GeoSeriesAccessor
sdf = pd.DataFrame.spatial.from_featureclass(fc)
sdf.head()

In [ ]:
sdf.spatial.plot(m1)